### Dataset & DataLoader 살펴보기
- PyTorch 에서 배치크기만 데이터를 조절하기 위한 메커니즘
- Dataset : 사용 데이터를 기반으로 사용자 정의 클래스 작성
- DataLoader : 지정된 Dataset 에서 지정된 batch size 만큼 피쳐와 타겟을 추출하여 전달

(1) 모듈 로딩 및 데이터 준비

In [1]:
### => 모듈 로딩
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
### 데이터 준비
x_data = torch.IntTensor([[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60], [50, 60, 70]])
y_data = torch.IntTensor([[20], [30], [40], [50], [60]])

print(f'x_data: {x_data.shape} {x_data.ndim}D')
print(f'y_data: {y_data.shape} {y_data.ndim}D')

x_data: torch.Size([5, 3]) 2D
y_data: torch.Size([5, 1]) 2D


(2) 데이터셋 생성

(2-1) TensorDataset 활용 : Dataset의 sub_class

In [3]:
### TensorDataset 클래스 로딩
from torch.utils.data import TensorDataset

In [4]:
dataset = TensorDataset(x_data, y_data)     # x_data, y_data 의 길이가 같아야 한다.
dataset

In [5]:
dataset.tensors

(tensor([[10, 20, 30],
         [20, 30, 40],
         [30, 40, 50],
         [40, 50, 60],
         [50, 60, 70]], dtype=torch.int32),
 tensor([[20],
         [30],
         [40],
         [50],
         [60]], dtype=torch.int32))

In [6]:
### __getitem__() 메서드 호출
dataset[0]

(tensor([10, 20, 30], dtype=torch.int32), tensor([20], dtype=torch.int32))

In [7]:
len(dataset)

5

(2-2) 사용자 정의 데이터셋 생성

In [8]:
### 데이터 준비
filename = '../data/iris.csv'
irisDF = pd.read_csv(filename)
irisDF.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [9]:
irisNP = np.loadtxt(filename, delimiter=',', skiprows=1, usecols=[0, 1, 2, 3])
irisNP[:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [10]:
### 데이터의 타입 체크
irisDF.__class__.__name__, irisNP.__class__.__name__

('DataFrame', 'ndarray')

In [11]:
if irisDF.__class__.__name__ == 'DataFrame':
    print('DF')
else:
    print('-----')

DF


In [12]:
### 데이터의 타입 체크
isinstance(irisDF, pd.DataFrame), isinstance(irisNP, pd.DataFrame)

(True, False)

In [13]:
isinstance([10], list), isinstance({10}, list)

(True, False)

In [53]:
### 사용자정의 Dataset 클래스
class DLDataset(Dataset):
    
    # 초기화 함수
    def __init__(self, x_data, y_data):
        super().__init__()
        
        # x, y 데이터가 DF라면 ndarray로 바꾸고, ndarray라면 그대로 저장
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        # ndarray -> tensor
        self.feature = torch.LongTensor(x_data)     # Long
        self.target = torch.LongTensor(y_data)      # Long
    
    # 데이터셋의 개수 체크 함수 콜백함수 (callback function)
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터 + 라벨 반환 콜백함수 (callback function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

In [54]:
### 피처와 라벨로 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetSR = irisDF[irisDF.columns[-1]]

In [55]:
### object 타입 타겟 -> int 타입 타겟 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetSR)
targetNP = targetNP.reshape(-1, 1)      # 1차원 -> 2차원으로 변환

In [56]:
# 데이터셋 생성 => DF, NP
my_dataset = DLDataset(featureDF, targetNP)

In [57]:
my_dataset[0], featureDF.iloc[0], targetNP[0]

((tensor([5, 3, 1, 0]), tensor([0])),
 sepal_length    5.1
 sepal_width     3.5
 petal_length    1.4
 petal_width     0.2
 Name: 0, dtype: float64,
 array([0]))

In [58]:
# 데이터셋 생성 => NP, NP
my_dataset2 = DLDataset(irisNP, targetNP)

In [59]:
my_dataset2[0]

(tensor([5, 3, 1, 0]), tensor([0]))

(2-3) 학습용, 검증용, 테스트용 Dataset

In [60]:
### 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트 데이터 비율
trainDS, validDS, testDS = random_split(my_dataset2, [0.7, 0.1, 0.2], generator=torch.Generator().manual_seed(42))

In [61]:
type(trainDS)

torch.utils.data.dataset.Subset

In [62]:
print(f'(train) Subset 속성=>\nindices : {trainDS.indices}\ndataset : {trainDS.dataset}')

(train) Subset 속성=>
indices : [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12]
dataset : <__main__.DLDataset object at 0x00000212C4563B20>


In [63]:
len(trainDS), len(validDS), len(testDS)

(105, 15, 30)

(3) DataLoader 생성 : 학습용, 검증용, 테스트용

In [64]:
# DataLoader 생성
# drop_last 매개변수 : 배치사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 (기본 : False, 마지막 미니배치도 사용)
batch_size = 10
trainDL = DataLoader(trainDS, batch_size=batch_size, drop_last=True)
validDL = DataLoader(validDS, batch_size=batch_size)
testDL = DataLoader(testDS, batch_size=batch_size)

len(trainDL), len(validDL), len(testDL)

(10, 2, 3)

In [65]:
# Epoch 당 반복 단위
print(f'batch_size: {batch_size}')
print(f'trainDS : {len(trainDS)}, validDS : {len(validDS)}, testDS : {len(testDS)}')
print(f'trainDL : {len(trainDL)}, validDS : {len(validDL)}, testDS : {len(testDL)}')

batch_size: 10
trainDS : 105, validDS : 15, testDS : 30
trainDL : 10, validDS : 2, testDS : 3


In [66]:
# DataLoader 
for _, (feature, target) in enumerate(trainDL):
    print(f'[{_}] feature {feature.shape} target {target.shape}')

[0] feature torch.Size([10, 4]) target torch.Size([10, 1])
[1] feature torch.Size([10, 4]) target torch.Size([10, 1])
[2] feature torch.Size([10, 4]) target torch.Size([10, 1])
[3] feature torch.Size([10, 4]) target torch.Size([10, 1])
[4] feature torch.Size([10, 4]) target torch.Size([10, 1])
[5] feature torch.Size([10, 4]) target torch.Size([10, 1])
[6] feature torch.Size([10, 4]) target torch.Size([10, 1])
[7] feature torch.Size([10, 4]) target torch.Size([10, 1])
[8] feature torch.Size([10, 4]) target torch.Size([10, 1])
[9] feature torch.Size([10, 4]) target torch.Size([10, 1])


(4) Model 클래스 정의 : 입/출력 피쳐수, 층 수, 은닉층의 노드 수
- 구조 설계
    - 입력층 : 입력 <= 피쳐 개수 (iris : 4개)
    - 은닉층 : 마음대로, 알아서 잘
    - 출력층 : 출력 <= [분류] 타겟 클래스 개수 [회귀] 1개

In [67]:
# 모델 클래스 정의
# 클래스명 : CModel
class CModel(torch.nn.Module):
    # 구성요소 정의 함수
    def __init__(self, in_, out_):
        super().__init__()      # 부모 클래스 생성자
        self.layer1 = torch.nn.Linear(in_, 10)
        self.relu = torch.nn.ReLU()
        self.layer2 = torch.nn.Linear(10, 6)
        self.layer3 = torch.nn.Linear(6, out_)
    # 순방향 학습 진행 함수
    def forward(self, x):
        x = self.layer1(x)      # W1X1 + W2X2 + ... + WnXn + b 10개 반환
        x = self.relu(x)        # relu() 결과 10개 반환
        x = self.layer2(x)      # W1X1 + W2X2 + ... + WnXn + b 6개 반환
        x = self.relu(x)        # relu() 결과 6개 반환
        x = self.layer3(x)      # W1X1 + W2X2 + ... + WnXn + b 3개 반환
        return x

(5) 학습 준비 : 실행 디바이스, 모델, 최적화, 손실함수, 학습회수 

In [68]:
## 실행 디바이스 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습 횟수
EPOCHS = 50

In [69]:
my_dataset.feature.shape

torch.Size([150, 4])

In [70]:
## 모델 인스턴스 생성
IN, OUT = my_dataset.feature.shape[1], len(np.unique(targetNP))
model = CModel(IN, OUT).to(DEVICE)
print(f'모델 구조 확인 : {model}')

모델 구조 확인 : CModel(
  (layer1): Linear(in_features=4, out_features=10, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=10, out_features=6, bias=True)
  (layer3): Linear(in_features=6, out_features=3, bias=True)
)


In [71]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1698, -0.3853,  0.4696,  0.4968],
        [-0.3809, -0.3539, -0.2186,  0.4207],
        [-0.1898,  0.4500,  0.3733, -0.3592],
        [-0.4405, -0.4455,  0.4082,  0.3087],
        [ 0.2938, -0.4734, -0.1356, -0.0548],
        [-0.4763, -0.1307,  0.3692, -0.2042],
        [-0.0108,  0.3270, -0.4327, -0.3455],
        [ 0.1968,  0.4198,  0.3467, -0.0590],
        [-0.0617,  0.4730, -0.1910,  0.3238],
        [ 0.4456,  0.1236, -0.2633, -0.1689]], requires_grad=True)
Parameter containing:
tensor([ 0.2848,  0.1726, -0.0781,  0.4318,  0.2814, -0.3171, -0.3917, -0.3410,
        -0.0625, -0.4712], requires_grad=True)
Parameter containing:
tensor([[-0.2010, -0.3022,  0.0123, -0.0337, -0.3098,  0.1109,  0.0592,  0.1005,
         -0.0161,  0.0914],
        [ 0.2965,  0.0620, -0.1508, -0.1005,  0.1407, -0.1877, -0.1995, -0.0677,
          0.3111, -0.2209],
        [ 0.2329,  0.2951,  0.0077, -0.0984, -0.2713, -0.1318, -0.3049, -0.1859,
         -0.0607,  0.0353],

In [72]:
# 손실함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(model.parameters())

* 학습 및 검증관련 함수 정의

In [77]:
## 학습 진행함수
def training():
    # 학습모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
    model.train()
    
    # 배치크기만큼 학습 진행 및 저장
    train_loss = []
    for cnt, (x, y) in enumerate(trainDL):
        # 배치크기만큼의 학습 데이터 준비
        feature, target = x.to(DEVICE), y.to(DEVICE)
        
        
        # 학습
        pre_target = model(feature)
        print(f'pre_target => {pre_target.shape}, {pre_target.ndim}D')
        print(f'target => {target.shape}, {target.ndim}D')
        
        # 손실계산
        loss = LOSS_FN(pre_target, target.squeeze())
        train_loss.append(loss)
        # W, b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()
    
    # 학습 진행 메시지 출력
    print(f'[Train loss] {loss}')
    
    return train_loss

In [78]:
## 검증 및 평가 진행함수
def testing():
    pass

In [79]:
## 예측 함수
def predict():
    pass

(6) 학습진행

In [80]:
for eps in range(EPOCHS):
    # 학습
    train_loss = training()
    #검증
    testing()
    print(f'[{eps}/{EPOCHS}] {sum(train_loss)/len(train_loss)}]')

RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float